In [1]:
class Node:
    def __init__(self, x, y, z, node_id, curr_strength):
        self.x = x
        self.y = y
        self.z = z
        self.curr_strength = curr_strength
        self.node_id = node_id
        self.signal_strengths = {}
        self.radius = 10

    def set_signal_strength(self, distance, strength):
        self.signal_strengths[distance] = strength

    def set_radius(self, rad):
        self.radius = rad
    
    def set_curr_strength(self, strength):
        self.curr_strength = strength
        
    def get_curr_strength(self):
        return self.curr_strength
    
    def get_node_id(self):
        return self.node_id
        
    def get_node_location(self):
        return [self.x, self.y, self.z]
        
    def get_radius(self):
        return self.radius
        
    def get_signal_strength(self, distance):
        return self.signal_strengths.get(distance, None)

    def draw(self, screen):
        pygame.draw.circle(screen, (255, 255, 255), (self.x, self.y), self.radius)

    def is_clicked(self, pos):
        return (self.x - pos[0])**2 + (self.y - pos[1])**2 <= self.radius**2

In [2]:
import random
import numpy as np
from scipy.optimize import least_squares
#Lets have our coordinate system be a 100x100x100 3d space.
#Generating random nodes for testing.
#This will create a 'real' location, then create X nodes that have a set signal strength away
#If noise is true, it will add noise to the calculation for a more rough estimate of location
def generate_random_nodes(numNodes, noise = False):
    real_location = np.random.randint(0, 100, 3)
    nodes = []
    for i in range(numNodes):
        node_loc = np.random.randint(0, 100, size=3)
        err = 0
        if noise:
            err = np.random.normal(0.0, 0.5)
        distance = np.linalg.norm(real_location - node_loc) + err
        new_node = Node(x = node_loc[0], y = node_loc[1], z = node_loc[2], node_id = i, curr_strength = distance)
        nodes.append(new_node)
    return nodes, real_location
    
def trilateration(nodes, distances):
    """
    Find the location of a point based on distances from known nodes.
    
    :param nodes: List of nested arrays [[x, y, z], ...] representing the coordinates of the nodes.
    :param distances: List of distances from each node to the unknown point.
    :return: Estimated coordinates of the unknown point.
    """
    def equations(point):
        x, y, z = point
        return [((x - nx)**2 + (y - ny)**2 + (z - nz)**2 - d**2) for [nx, ny, nz], d in zip(nodes, distances)]

    # Initial guess for the unknown point
    initial_guess = [0, 0, 0]

    result = least_squares(equations, initial_guess)

    return result.x
    
def main():
    nodes, real_loc = generate_random_nodes(8, noise = True)
    
    print(real_loc)
    xs = []
    ys = []
    zs = []
    locs = []
    dists = []
    for i in range(len(nodes)):
        locations = nodes[i].get_node_location()
        xs.append(locations[0])
        ys.append(locations[1])
        zs.append(locations[2])
        locs.append(locations)
        dists.append(nodes[i].get_curr_strength())
    
    #Time for math!
    #Example usage
    print(locs)
    print(dists)
    guess = trilateration(locs, dists)
    print(guess)

In [3]:
main()

[56 16 78]
[[96, 39, 58], [53, 60, 61], [13, 74, 93], [1, 41, 93], [99, 30, 83], [26, 17, 48], [33, 42, 14], [54, 41, 12]]
[50.32416144150654, 47.79847655645206, 74.11804609463714, 62.2279903848817, 45.55793980675036, 42.8812981081501, 72.9602429091484, 71.0349997947039]
[56.06133495 15.62478931 78.1982926 ]
